In [1]:
# Google Colabと自分のパソコンの両方で使えるようにするための工夫

using Pkg

"""すでにPkg.add済みのパッケージのリスト (高速化のために用意)"""
_packages_added = [sort!(readdir(Sys.STDLIB));
    [info.name for (uuid, info) in Pkg.dependencies() if info.is_direct_dep]]

"""_packages_added内にないパッケージをPkg.addする"""
add_pkg_if_not_added_yet(pkg) = if !(pkg in _packages_added)
    println(stderr, "# $(pkg).jl is not added yet, so let's add it.")
    Pkg.add(pkg)
end

"""expr::Exprからusing内の`.`を含まないモジュール名を抽出"""
function find_using_pkgs(expr::Expr)
    pkgs = String[]
    function traverse(expr::Expr)
        if expr.head == :using
            for arg in expr.args
                if arg.head == :. && length(arg.args) == 1
                    push!(pkgs, string(arg.args[1]))
                elseif arg.head == :(:) && length(arg.args[1].args) == 1
                    push!(pkgs, string(arg.args[1].args[1]))
                end
            end
        else
            for arg in expr.args arg isa Expr && traverse(arg) end
        end
    end
    traverse(expr)
    pkgs
end

"""必要そうなPkg.addを追加するマクロ"""
macro autoadd(expr)
    pkgs = find_using_pkgs(expr)
    :(add_pkg_if_not_added_yet.($(pkgs)); $expr)
end

@autoadd

In [2]:
macro usingdocstringtranslation()
    quote
        translationurl = "https://github.com/AtelierArith/DocStrBankExperimental.jl/releases/download/full-ja/translation.zip"
        #translationurl = "https://github.com/AtelierArith/DocStrBankExperimental.jl/releases/download/stats-ja/translation.zip"
        scratchspaces_dir = joinpath(DEPOT_PATH[1], "scratchspaces", "d404e13b-1f8e-41a5-a26a-0b758a0c6c97")
        translation_dir = joinpath(scratchspaces_dir, "translation")
        if isdir(translation_dir)
            println(stderr, "Directory $translation_dir already exists.")
            println(stderr, "To download and extract translation.zip again, the directory must be deleted.")
        else
            println(stderr, "translation.zip to be downloaded will be extracted into $translation_dir.")
            run(`wget --no-verbose $(translationurl) -P $(scratchspaces_dir)`)
            run(`unzip -q $(joinpath(scratchspaces_dir, "translation.zip")) -d $(scratchspaces_dir)`)
        end
        @autoadd using DocstringTranslation
    end
end

# Google Colabで次の行の実行には3分秒程度かかる。
@usingdocstringtranslation; @switchlang! :ja

Directory D:\.julia\scratchspaces\d404e13b-1f8e-41a5-a26a-0b758a0c6c97\translation already exists.
To download and extract translation.zip again, the directory must be deleted.
┌ Warning: The OPENAI_API_KEY has not been set. Please set OPENAI_API_KEY to the environment variable to use the translation function.
└ @ DocstringTranslation D:\.julia\packages\DocstringTranslation\0YgZP\src\switchlang.jl:26


"ja"

In [3]:
?range

search: range logrange angle rand randn LinRange



```
range(start, stop, length)
range(start, stop; length, step)
range(start; length, stop, step)
range(;start, length, stop, step)
```

引数から均等に間隔を空けた要素を持つ特化型配列（[`AbstractRange`](@ref)）を構築します。数学的には、範囲は `start`、`step`、`stop`、`length` のいずれか3つによって一意に決まります。範囲の有効な呼び出しは次のとおりです。

  * `start`、`step`、`stop`、`length` のいずれか3つを指定して `range` を呼び出します。
  * `start`、`stop`、`length` のいずれか2つを指定して `range` を呼び出します。この場合、`step` は1であると仮定されます。両方の引数が整数の場合、[`UnitRange`](@ref) が返されます。
  * `stop` または `length` のいずれか1つを指定して `range` を呼び出します。この場合、`start` と `step` は1であると仮定されます。

返される型に関する詳細は拡張ヘルプを参照してください。また、対数間隔の点については [`logrange`](@ref) も参照してください。

# 例

```jldoctest
julia> range(1, length=100)
1:100

julia> range(1, stop=100)
1:100

julia> range(1, step=5, length=100)
1:5:496

julia> range(1, step=5, stop=100)
1:5:96

julia> range(1, 10, length=101)
1.0:0.09:10.0

julia> range(1, 100, step=5)
1:5:96

julia> range(stop=10, length=5)
6:10

julia> range(stop=10, step=1, length=5)
6:1:10

julia> range(start=1, step=1, stop=10)
1:1:10

julia> range(; length = 10)
Base.OneTo(10)

julia> range(; stop = 6)
Base.OneTo(6)

julia> range(; stop = 6.5)
1.0:1.0:6.0
```

`length` が指定されておらず、`stop - start` が `step` の整数倍でない場合、`stop` の手前で終了する範囲が生成されます。

```jldoctest
julia> range(1, 3.5, step=2)
1.0:2.0:3.0
```

中間値が合理的に計算されるように特別な配慮がなされています。この誘発されたオーバーヘッドを避けるためには、[`LinRange`](@ref) コンストラクタを参照してください。

!!! compat "Julia 1.1"
    位置引数としての `stop` は少なくとも Julia 1.1 が必要です。


!!! compat "Julia 1.7"
    キーワード引数なしのバージョンと `start` をキーワード引数とするバージョンは少なくとも Julia 1.7 が必要です。


!!! compat "Julia 1.8"
    `stop` を唯一のキーワード引数、または `length` を唯一のキーワード引数とするバージョンは少なくとも Julia 1.8 が必要です。


# 拡張ヘルプ

引数が整数の場合、`range` は次のように `Base.OneTo` を生成します。

  * `length` のみが提供されている場合
  * `stop` のみが提供されている場合

引数が整数の場合、`range` は次のように `UnitRange` を生成します。

  * `start` と `stop` のみが提供されている場合
  * `length` と `stop` のみが提供されている場合

`step` が提供されている場合、たとえ1として指定されていても `UnitRange` は生成されません。


In [4]:
??range

search: range logrange angle rand randn LinRange



```
range(start, stop, length)
range(start, stop; length, step)
range(start; length, stop, step)
range(;start, length, stop, step)
```

引数から均等に間隔を空けた要素を持つ特化型配列（[`AbstractRange`](@ref)）を構築します。数学的には、範囲は `start`、`step`、`stop`、`length` のいずれか3つによって一意に決まります。範囲の有効な呼び出しは次のとおりです。

  * `start`、`step`、`stop`、`length` のいずれか3つを指定して `range` を呼び出します。
  * `start`、`stop`、`length` のいずれか2つを指定して `range` を呼び出します。この場合、`step` は1であると仮定されます。両方の引数が整数の場合、[`UnitRange`](@ref) が返されます。
  * `stop` または `length` のいずれか1つを指定して `range` を呼び出します。この場合、`start` と `step` は1であると仮定されます。

返される型に関する詳細は拡張ヘルプを参照してください。また、対数間隔の点については [`logrange`](@ref) も参照してください。

# 例

```jldoctest
julia> range(1, length=100)
1:100

julia> range(1, stop=100)
1:100

julia> range(1, step=5, length=100)
1:5:496

julia> range(1, step=5, stop=100)
1:5:96

julia> range(1, 10, length=101)
1.0:0.09:10.0

julia> range(1, 100, step=5)
1:5:96

julia> range(stop=10, length=5)
6:10

julia> range(stop=10, step=1, length=5)
6:1:10

julia> range(start=1, step=1, stop=10)
1:1:10

julia> range(; length = 10)
Base.OneTo(10)

julia> range(; stop = 6)
Base.OneTo(6)

julia> range(; stop = 6.5)
1.0:1.0:6.0
```

`length` が指定されておらず、`stop - start` が `step` の整数倍でない場合、`stop` の手前で終了する範囲が生成されます。

```jldoctest
julia> range(1, 3.5, step=2)
1.0:2.0:3.0
```

中間値が合理的に計算されるように特別な配慮がなされています。この誘発されたオーバーヘッドを避けるためには、[`LinRange`](@ref) コンストラクタを参照してください。

!!! compat "Julia 1.1"
    位置引数としての `stop` は少なくとも Julia 1.1 が必要です。


!!! compat "Julia 1.7"
    キーワード引数なしのバージョンと `start` をキーワード引数とするバージョンは少なくとも Julia 1.7 が必要です。


!!! compat "Julia 1.8"
    `stop` を唯一のキーワード引数、または `length` を唯一のキーワード引数とするバージョンは少なくとも Julia 1.8 が必要です。


# 拡張ヘルプ

引数が整数の場合、`range` は次のように `Base.OneTo` を生成します。

  * `length` のみが提供されている場合
  * `stop` のみが提供されている場合

引数が整数の場合、`range` は次のように `UnitRange` を生成します。

  * `start` と `stop` のみが提供されている場合
  * `length` と `stop` のみが提供されている場合

`step` が提供されている場合、たとえ1として指定されていても `UnitRange` は生成されません。


In [5]:
range(length=10)

Base.OneTo(10)

In [6]:
"""2倍する関数"""
double(x) = 2x

double

In [7]:
?double

search: double Cdouble NTuple tuple showable Tuple Module ntuple module



2倍する関数


In [8]:
@autoadd begin
using LinearAlgebra
using Distributions
using StatsPlots
using StatsBase
using StatsFuns
end

In [9]:
?dot

search: dot do cot dof det stdout @doc plot coth docs font cotd acot sort edit



```
dot(x, y)
x ⋅ y
```

2つのベクトルのドット積を計算します。複素ベクトルの場合、最初のベクトルは共役されます。

`dot`は、要素に対して`dot`が定義されている限り、任意の次元の配列を含む任意の反復可能オブジェクトでも機能します。

`dot`は、引数が等しい長さでなければならないという追加の制約を持ちながら、`sum(dot(vx,vy) for (vx,vy) in zip(x, y))`と意味的に同等です。

`x ⋅ y`（ここで`⋅`はREPLで`\cdot`をタブ補完することで入力できます）は`dot(x, y)`の同義語です。

# 例

```jldoctest
julia> dot([1; 1], [2; 3])
5

julia> dot([im; im], [1; 1])
0 - 2im

julia> dot(1:5, 2:6)
70

julia> x = fill(2., (5,5));

julia> y = fill(3., (5,5));

julia> dot(x, y)
150.0
```

---

```
dot(x, A, y)
```

ベクトル `x` と `y` の間で、`A*y` の中間結果を保存せずに一般化された内積 `dot(x, A*y)` を計算します。二引数の [`dot(_,_)`](@ref) と同様に、再帰的に動作します。さらに、複素ベクトルの場合、最初のベクトルは共役になります。

!!! compat "Julia 1.4"
    三引数の `dot` は少なくとも Julia 1.4 が必要です。


# 例

```jldoctest
julia> dot([1; 1], [1 2; 3 4], [2; 3])
26

julia> dot(1:5, reshape(1:25, 5, 5), 2:6)
4850

julia> ⋅(1:5, reshape(1:25, 5, 5), 2:6) == dot(1:5, reshape(1:25, 5, 5), 2:6)
true
```


In [10]:
?Gamma

search: Gamma harmmean gamutmax gammapdf gammacdf



```
Gamma(α,θ)
```

*ガンマ分布*は、形状パラメータ`α`とスケール`θ`を持ち、確率密度関数は次のようになります。

$$
f(x; \alpha, \theta) = \frac{x^{\alpha-1} e^{-x/\theta}}{\Gamma(\alpha) \theta^\alpha},
\quad x > 0
$$

```julia
Gamma()          # 形状が単位、スケールが単位のガンマ分布、すなわちGamma(1, 1)
Gamma(α)         # 形状がα、スケールが単位のガンマ分布、すなわちGamma(α, 1)
Gamma(α, θ)      # 形状がα、スケールがθのガンマ分布

params(d)        # パラメータを取得、すなわち(α, θ)
shape(d)         # 形状パラメータを取得、すなわちα
scale(d)         # スケールパラメータを取得、すなわちθ
```

外部リンク

  * [ガンマ分布 - Wikipedia](http://en.wikipedia.org/wiki/Gamma_distribution)


In [11]:
?density

search: density density! cdensity cdensity! identity endswith Registry



```
density(x)
density!(x)
```

xのカーネル密度推定の線グラフを作成します。密度プロットの滑らかさは`bandwidth`（実数の正の数）によって定義されます。

# 引数

  * `x`: 確率密度推定のためのサンプルのAbstractVector

# キーワード引数

  * `trim`::Bool: 分布の尾を切り取ることを有効にします。
  * `bandwidth`::Number: 低いバンド幅はアンダースムージングを引き起こし、高いバンド幅はオーバースムージングを引き起こします。

# 例

```julia-repl
julia> density(randn(100), bandwidth = -0.01, trim = false)
output : ERROR: Bandwidth must be positive

julia> density(randn(100), bandwidth = 0.1, trim = false)  # 極端さとアンダースムージングを持つ曲線
julia> density(randn(100), bandwidth = 10, trim = true)  # 極端さのないオーバースムージングの曲線
```

# 例

```julia-repl
julia> using StatsPlots
julia> density(randn(100_000))
```


In [12]:
?ecdf

search: ecdf ccdf cdf betacdf bvncdf eof betaccdf secd @df ecdfplot end modf



```
ecdf(X; weights::AbstractWeights)
```

与えられたサンプルのベクトル `X` に基づいて経験的累積分布関数 (ECDF) を返します。オプションで `weights` を提供すると、重み付き ECDF が返されます。

注意: この関数は呼び出し可能な合成型を返し、他のサンプルに対して CDF 値を評価するために適用できます。

`extrema`、`minimum`、および `maximum` は、関数が区間 $(0,1)$ 内にある範囲を取得するためにサポートされています; 関数は全実数直線に対して定義されています。


In [13]:
?logit

search: logit logdet logistic log1p log4π log10 log2π log2 logπ log logpdf



```julia
logit(x)

```

[logit](https://en.wikipedia.org/wiki/Logit)またはロジット変換は、次のように定義されます。

$$
\operatorname{logit}(x) = \log\left(\frac{x}{1-x}\right)
$$

$$
0 < x < 1
$$

のとき。

その逆は[`logistic`](@ref)関数です。
